<a href="https://colab.research.google.com/github/zboraon/proxy_impuation_SSVS/blob/main/KF_SSVS_imputation_proxy_with_all_data_kf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Necessary libraries

I recommend the users to use this specific model on Colab, or a Debian based distro. If you want to apply the same model on a different data set, you have to know how to install jags (https://sourceforge.net/projects/mcmc-jags/) and specific R packages to your computer.

Install and load the necessary libraries.


In [ ]:
system("apt install -y jags")
system("apt install -y r-base")

# system("apt install fonts-lmodern")
# system("apt install lmodern")
# system("apt install -y libcairo2-dev") # For Cairo graphics library


In [ ]:
if (!require("runjags")) install.packages("runjags")
if (!require("RCurl")) install.packages("RCurl")
if (!require("bayesplot")) install.packages("bayesplot")
if (!require("patchwork")) install.packages("patchwork")
if (!require("zip")) install.packages("zip")

#install.packages("extrafont")

In [ ]:
library(RCurl)
library(runjags)
library(coda)
library(parallel)

# for plots
library(bayesplot)
library(ggplot2)
library(dplyr)
library(patchwork)

# for saving
library(zip)

#library(extrafont)

# Import fonts, including 'lmodern' if available
#font_import(paths = NULL, prompt = FALSE)
#loadfonts(device = "pdf")

#font_import(pattern = "lmsans*", prompt = FALSE)
#loadfonts()
#par(family = "LM Sans 10")

# Preliminary data definitions
Predefine intervals, data, etc.

In [ ]:
# Filter the data for a specific time range
# Is your regression "forwards" or "backwards" in time?
# For example, if the interval to be imputed is 0--10 ka BP,
# and there is data between 10--50 ka BP, your approach is "forwards".
# You approach the missing inteval from 50 ka BP to the 10 ka BP. Otherwise,
# your approach is backwards.
timescale = "forwards"

# For the upper explanation, your whole interval is time_range <- c(50,0)
time_range <- c(624, 478) # choose the time interval for your whole regression
# c(335, 196) #c(533, 381)#c(381,533) #c(132, 1) #c(132, 1)
# c(80, 132) #c(624, 380)# c(80, 218) # c(80, 196) c(218, 132)

# temporal length for forecast. the final time point minus this value
# should correspond to the length of the data to be imputed
# for the upper example, 10-0+1=11, i.e. kyrs_to_forecast <- 11
kyrs_to_forecast <- 61


# resolution of the state-space model, depends on your choice. In this application,
# I chose it to be 1kyrs
resolution <- 1

# data with missing values. this name must be the same with the data
# that is used in the jags model. this code handles it for Ohrid data.
data_to_forecast <- "O18" #"O18"C13

# Specify the time points where you want to interpolate
# Adjust the time sequences based on the timescale
if (timescale == "forwards") {
  new_time <- seq(time_range[1], time_range[2], by = -1 * resolution)
  new_time_shrt <- seq(time_range[1], time_range[2] + kyrs_to_forecast, by = -1 * resolution)
} else {
  new_time <- seq(time_range[1], time_range[2], by = resolution)
  new_time_shrt <- seq(time_range[1], time_range[2] - kyrs_to_forecast, by = resolution)
}

# the names of the covariates, as in the data to forecast
covariatenames <- c("AP", "MS", "Qz", "TIC", "TOC", "TN", "TS", "Clay", "bSiO2", "Ca", "K", "Fe", "Zr")

# type of the model,
kf_model <- "onlyageerror"  # Modify as needed for different models: "noerror", "eiv" "fullstochastic" "onlyageerror"

# Select your regression model


In [ ]:
# this is to specify if your regression model is a regression model with
# AR(1), random walk level, static level, or a simple regression
# The models other than simplyssvs are not suitable due to the increased variance
# at forecasts, and they are against the logic of simply building the data with
# the rest of the data. In case you want to use them, uncomment the lines and
# if needed make necessary appropriate changes. I did not make the last changes
# on them. I am not sure if they are working.
mymodel <- "simplyssvs" # model = c("simplyssvs", "rwlevel", "staticlevel", "ar1")

# Read Ohrid data

In [ ]:
# Read data
ohrid_geochem_data_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/main/Data/",
  "Lake_Ohrid/Ohrid_geochemistry.csv",
  sep = ""
)
ohrid_MS_data_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/main/Data/",
  "Lake_Ohrid/Ohrid_MS.csv",
  sep = ""
)
ohrid_XRF_data_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/main/Data/",
  "Lake_Ohrid/Ohrid_XRF.csv",
  sep = ""
)
ohrid_pollen_data_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/main/Data/",
  "Lake_Ohrid/Ohrid_pollen.csv",
  sep = ""
)

ohrid_age_depth_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/main/Data/",
  "Lake_Ohrid/Ohrid_agedepth.csv",
  sep = ""
)

ohrid_XRF_data_francke_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/main/Data/",
  "Lake_Ohrid/Ohrid_XRF_franckeetal16.csv",
  sep = ""
)

ohrid_sediment_data_francke_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/main/Data/",
  "Lake_Ohrid/Ohrid_sediment_franckeetal16.csv",
  sep = ""
)

ohrid_geochem_url <- getURL(ohrid_geochem_data_url_text)
ohrid_ms_url <- getURL(ohrid_MS_data_url_text)
ohrid_xrf_url <- getURL(ohrid_XRF_data_url_text)
ohrid_pollen_url <- getURL(ohrid_pollen_data_url_text)
ohrid_agedepth_url <- getURL(ohrid_age_depth_url_text)
ohrid_xrf_francke_url <- getURL(ohrid_XRF_data_francke_url_text)
ohrid_sediment_data_francke_url <- getURL(ohrid_sediment_data_francke_url_text)

# Load data
ohrid_raw_geochem <- read.csv(text = ohrid_geochem_url)
ohrid_raw_ms <- read.csv(text = ohrid_ms_url)
ohrid_raw_xrf <- read.csv(text = ohrid_xrf_url)
ohrid_raw_pollen <- read.csv(text = ohrid_pollen_url)
ohrid_agedepth <- read.csv(text = ohrid_agedepth_url)
ohrid_raw_xrf_francke <- read.csv(text = ohrid_xrf_francke_url)
ohrid_raw_sediment_francke <- read.csv(text = ohrid_sediment_data_francke_url)

# Process the Ohrid data

In [ ]:
# Process the geochemical data
for (proxyname in c("Qz", "TIC", "TOC", "TN", "TS", "O18", "C13")) {
  # Dynamically create the column name to extract
  data <- na.omit(ohrid_raw_geochem[c("geochem_depth", proxyname)])

  # Dynamically assign the data to a new variable with the name "<proxyname>_data"
  assign(paste0(proxyname, "_data"), data)
}

# Process the pollen data
AP_data <- na.omit(ohrid_raw_pollen[c("pollen_depth", "AP")])
Quercus_data <- na.omit(ohrid_raw_pollen[c("pollen_depth", "Quercus")])

# Process the XRF data and divide them to 1000 as in Wagner et al. (2019)
K_data <- ohrid_raw_xrf %>%
  select(xrf_depth, K) %>%
  na.omit() %>%
  mutate(K = K / 1000)
Ca_data <- ohrid_raw_xrf %>%
  select(xrf_depth, Ca) %>%
  na.omit() %>%
  mutate(Ca = Ca / 1000)

# Process the magnetic susceptibility data
MS_data <- na.omit(ohrid_raw_ms[c("MS_depth", "MS")])

# Process the sediment (Francke) data
SR_data <- na.omit(ohrid_raw_sediment_francke[c("sediment_depth", "SR")])
Clay_data <- na.omit(ohrid_raw_sediment_francke[c("sediment_depth", "Clay")])
bSiO2_data <- na.omit(ohrid_raw_sediment_francke[c("sediment_depth", "bSiO2")])

# Process the XRF (Francke) data and divide Fe to 1000 as in Wagner et al. (2019)
Fe_data <- ohrid_raw_xrf_francke %>%
  select(xrf_francke_depth, Fe) %>%
  na.omit() %>%
  mutate(Fe = Fe / 1000)
Zr_data <- na.omit(ohrid_raw_xrf_francke[c("xrf_francke_depth", "Zr")])



# Calculate the t-distribution critical values for 6 df, see Christen and Perez 2010 abstract
t_84 <- qt(0.84, df = 6)  # 1-sigma quantile (84%)
t_975 <- qt(0.975, df = 6)  # 2-sigma quantile (97.5%)

# Scaling factor
scaling_factor <- t_84 / t_975

options(warn=-1)

# Loop over each covariate for geochem for interpolation of ages
for (proxyname in c("Qz", "TIC", "TOC", "TN", "TS", "O18", "C13")) {

  # Retrieve the data for the current proxy
  proxy_data <- get(paste0(proxyname, "_data"))

  # Apply the interpolation and calculations
  proxy_data <- proxy_data %>%
    mutate(min = approx(ohrid_agedepth$depth, ohrid_agedepth$min, xout = geochem_depth, rule = 2)$y,
           max = approx(ohrid_agedepth$depth, ohrid_agedepth$max, xout = geochem_depth, rule = 2)$y,
           median = approx(ohrid_agedepth$depth, ohrid_agedepth$median, xout = geochem_depth, rule = 2)$y,
           agekaBP = approx(ohrid_agedepth$depth, ohrid_agedepth$wmean, xout = geochem_depth, rule = 2)$y,
           uncertainty_1sigma = (max - min) / 2 * scaling_factor,
           measurement_uncertainty = 0.01 * get(proxyname)
    )

  # Assign the updated data back to the environment with the name "<proxyname>_data"
  assign(paste0(proxyname, "_data"), proxy_data)
}

# Loop over each covariate for pollen for interpolation of ages
for (proxyname in c("AP", "Quercus")) {

  # Retrieve the data for the current proxy
  proxy_data <- get(paste0(proxyname, "_data"))

  # Apply the interpolation and calculations
  proxy_data <- proxy_data %>%
    mutate(min = approx(ohrid_agedepth$depth, ohrid_agedepth$min, xout = pollen_depth, rule = 2)$y,
           max = approx(ohrid_agedepth$depth, ohrid_agedepth$max, xout = pollen_depth, rule = 2)$y,
           median = approx(ohrid_agedepth$depth, ohrid_agedepth$median, xout = pollen_depth, rule = 2)$y,
           agekaBP = approx(ohrid_agedepth$depth, ohrid_agedepth$wmean, xout = pollen_depth, rule = 2)$y,
           uncertainty_1sigma = (max - min) / 2 * scaling_factor,
           measurement_uncertainty = 0.01 * get(proxyname)
    )

  # Assign the updated data back to the environment with the name "<proxyname>_data"
  assign(paste0(proxyname, "_data"), proxy_data)
}

# Loop over each covariate for xrf  for interpolation of ages
for (proxyname in c("Ca", "K")) {

  # Retrieve the data for the current proxy
  proxy_data <- get(paste0(proxyname, "_data"))

  # Apply the interpolation and calculations
  proxy_data <- proxy_data %>%
    mutate(min = approx(ohrid_agedepth$depth, ohrid_agedepth$min, xout = xrf_depth, rule = 2)$y,
           max = approx(ohrid_agedepth$depth, ohrid_agedepth$max, xout = xrf_depth, rule = 2)$y,
           median = approx(ohrid_agedepth$depth, ohrid_agedepth$median, xout = xrf_depth, rule = 2)$y,
           agekaBP = approx(ohrid_agedepth$depth, ohrid_agedepth$wmean, xout = xrf_depth, rule = 2)$y,
           uncertainty_1sigma = (max - min) / 2 * scaling_factor,
           measurement_uncertainty = 0.01 * get(proxyname)
    )

  # Assign the updated data back to the environment with the name "<proxyname>_data"
  assign(paste0(proxyname, "_data"), proxy_data)
}

# Interpolate values for MS data depths
MS_data <- MS_data %>%
  mutate(min = approx(ohrid_agedepth$depth, ohrid_agedepth$min, xout = MS_depth, rule = 2)$y,
         max = approx(ohrid_agedepth$depth, ohrid_agedepth$max, xout = MS_depth, rule = 2)$y,
         median = approx(ohrid_agedepth$depth, ohrid_agedepth$median, xout = MS_depth, rule = 2)$y,
         agekaBP = approx(ohrid_agedepth$depth, ohrid_agedepth$wmean, xout = MS_depth, rule = 2)$y,
         uncertainty_1sigma = (max - min) / 2 * scaling_factor,
         measurement_uncertainty = 0.01 * MS_data$MS
  )

# Loop over each covariate for sediment  for interpolation of ages
for (proxyname in c("Clay", "bSiO2")) { #"SR",

  # Retrieve the data for the current proxy
  proxy_data <- get(paste0(proxyname, "_data"))

  # Apply the interpolation and calculations
  proxy_data <- proxy_data %>%
    mutate(min = approx(ohrid_agedepth$depth, ohrid_agedepth$min, xout = sediment_depth, rule = 2)$y,
           max = approx(ohrid_agedepth$depth, ohrid_agedepth$max, xout = sediment_depth, rule = 2)$y,
           median = approx(ohrid_agedepth$depth, ohrid_agedepth$median, xout = sediment_depth, rule = 2)$y,
           agekaBP = approx(ohrid_agedepth$depth, ohrid_agedepth$wmean, xout = sediment_depth, rule = 2)$y,
           uncertainty_1sigma = (max - min) / 2 * scaling_factor,
           measurement_uncertainty = 0.01 * get(proxyname)
    )

  # Assign the updated data back to the environment with the name "<proxyname>_data"
  assign(paste0(proxyname, "_data"), proxy_data)
}

# Loop over each covariate for XRF (francke)  for interpolation of ages
for (proxyname in c("Fe", "Zr")) {

  # Retrieve the data for the current proxy
  proxy_data <- get(paste0(proxyname, "_data"))

  # Apply the interpolation and calculations
  proxy_data <- proxy_data %>%
    mutate(min = approx(ohrid_agedepth$depth, ohrid_agedepth$min, xout = xrf_francke_depth, rule = 2)$y,
           max = approx(ohrid_agedepth$depth, ohrid_agedepth$max, xout = xrf_francke_depth, rule = 2)$y,
           median = approx(ohrid_agedepth$depth, ohrid_agedepth$median, xout = xrf_francke_depth, rule = 2)$y,
           agekaBP = approx(ohrid_agedepth$depth, ohrid_agedepth$wmean, xout = xrf_francke_depth, rule = 2)$y,
           uncertainty_1sigma = (max - min) / 2 * scaling_factor,
           measurement_uncertainty = 0.01 * get(proxyname)
    )

  # Assign the updated data back to the environment with the name "<proxyname>_data"
  assign(paste0(proxyname, "_data"), proxy_data)
}

# Listing the data

In [ ]:
# Filter the data for a specific time range
timescale = timescale
time_range <- time_range
kyrs_to_forecast <- kyrs_to_forecast
resolution <- resolution # 0.5
data_to_forecast <- data_to_forecast #"O18"
# Specify the time points where you want to interpolate
# Adjust the time sequences based on the timescale
if (timescale == "forwards") {
  new_time <- seq(time_range[1], time_range[2], by = -1 * resolution)
  new_time_shrt <- seq(time_range[1], time_range[2] + kyrs_to_forecast, by = -1 * resolution)
} else {
  new_time <- seq(time_range[1], time_range[2], by = resolution)
  new_time_shrt <- seq(time_range[1], time_range[2] - kyrs_to_forecast, by = resolution)
}

# List of datasets
datasets <- lapply(covariatenames, function(name) get(paste0(name, "_data")))


# Corresponding variable names
var_names <- covariatenames

# Corresponding proxy values and IDs
proxy_values <- var_names
proxy_ids <- 1:length(var_names)

# Create cropped datasets with dynamic names
if (timescale == "forwards") {
  for (i in seq_along(datasets)) {
    data <- datasets[[i]]
    var_name <- var_names[i]
    cropped_data <- data[data$agekaBP < time_range[1] & data$agekaBP > time_range[2], ]
    dynamic_name <- paste0(var_name, "_cropped")
    assign(dynamic_name, cropped_data)
  }
} else {
  for (i in seq_along(datasets)) {
    data <- datasets[[i]]
    var_name <- var_names[i]
    cropped_data <- data[data$agekaBP < time_range[2] & data$agekaBP > time_range[1], ]
    dynamic_name <- paste0(var_name, "_cropped")
    assign(dynamic_name, cropped_data)
  }
}

# Function to compute the centered log-ratio (CLR) transformation
clr_transform <- function(data_matrix, pseudocount = 1e-10) {
  # Ensure the input is a matrix
  if (!is.matrix(data_matrix)) {
    stop("Input must be a matrix.")
  }

  # Replace zeros with the pseudocount
  data_matrix[data_matrix == 0] <- pseudocount

  # Compute the geometric mean for each row
  geometric_mean <- apply(data_matrix, 1, function(row) exp(mean(log(row))))

  # Perform CLR transformation: subtract log of the geometric mean from log of each element
  clr_matrix <- log(data_matrix) - log(geometric_mean)

  # Return the CLR-transformed matrix
  return(clr_matrix)
}

# Keep published data
Ca_cropped$Ca_published <- Ca_cropped$Ca
Fe_cropped$Fe_published <- Fe_cropped$Fe
K_cropped$K_published <- K_cropped$K
Zr_cropped$Zr_published <- Zr_cropped$Zr

# Treat XRF data as compositional, and transform to clr
XRF_cropped <- cbind(Ca_cropped$Ca_published, Fe_cropped$Fe_published, K_cropped$K_published, Zr_cropped$Zr_published)
XRF_cropped_clr <- clr_transform(XRF_cropped)

# Assign transformed data
Ca_cropped$Ca <- XRF_cropped_clr[ ,1]
Fe_cropped$Fe <- XRF_cropped_clr[ ,2]
K_cropped$K <- XRF_cropped_clr[ ,3]
Zr_cropped$Zr <- XRF_cropped_clr[ ,4]

# Initialize empty vectors
covariates_agekaBP <- c()
covariates_age_uncertainty <- c()
covariates_proxy_value <- c()
covariates_proxy_uncertainty <- c()
covariates_proxy_id <- c()
covariates_time_idx <- c()

# Loop through each cropped dataset and extract the necessary values

if (timescale == "forwards") {
  # Sort new_time in increasing order
  new_time_sorted <- sort(new_time)

  for (i in seq_along(var_names)) {
    var_name <- var_names[i]
    proxy_value <- proxy_values[i]
    proxy_id <- proxy_ids[i]

    dynamic_name <- paste0(var_name, "_cropped")
    if (exists(dynamic_name)) {
      cropped_data <- get(dynamic_name)

      # Append data for each variable
      covariates_agekaBP <- c(covariates_agekaBP, cropped_data$agekaBP)
      covariates_age_uncertainty <- c(covariates_age_uncertainty, cropped_data$uncertainty_1sigma)
      covariates_proxy_value <- c(covariates_proxy_value, cropped_data[[proxy_value]])
      covariates_proxy_uncertainty <- c(covariates_proxy_uncertainty, cropped_data$measurement_uncertainty)
      covariates_proxy_id <- c(covariates_proxy_id, rep(proxy_id, nrow(cropped_data)))

      # Use findInterval with sorted new_time
      time_idx <- findInterval(cropped_data$agekaBP, new_time_sorted)

      # Reverse the indices for "forwards" case
      reversed_time_idx <- length(new_time_sorted) - time_idx + 1

      # Append the reversed indices
      covariates_time_idx <- c(covariates_time_idx, reversed_time_idx)
    }
  }
} else {
  # For backwards case, no sorting or reversing needed
  for (i in seq_along(var_names)) {
    var_name <- var_names[i]
    proxy_value <- proxy_values[i]
    proxy_id <- proxy_ids[i]

    dynamic_name <- paste0(var_name, "_cropped")
    if (exists(dynamic_name)) {
      cropped_data <- get(dynamic_name)

      # Append data for each variable
      covariates_agekaBP <- c(covariates_agekaBP, cropped_data$agekaBP)
      covariates_age_uncertainty <- c(covariates_age_uncertainty, cropped_data$uncertainty_1sigma)
      covariates_proxy_value <- c(covariates_proxy_value, cropped_data[[proxy_value]])
      covariates_proxy_uncertainty <- c(covariates_proxy_uncertainty, cropped_data$measurement_uncertainty)
      covariates_proxy_id <- c(covariates_proxy_id, rep(proxy_id, nrow(cropped_data)))

      # Find interval without reversing in the forwards case
      covariates_time_idx <- c(covariates_time_idx, findInterval(cropped_data$agekaBP, new_time))
    }
  }
}



# Combine the data with new indices
combined_data <- data.frame(
  covariates_agekaBP = covariates_agekaBP,
  covariates_age_uncertainty = covariates_age_uncertainty,
  covariates_proxy_value = covariates_proxy_value,
  covariates_proxy_uncertainty = covariates_proxy_uncertainty,
  covariates_proxy_id = covariates_proxy_id,
  covariates_time_idx = covariates_time_idx
  )

# Add the new column with corresponding covariate names
combined_data$covariate_name <- covariatenames[combined_data$covariates_proxy_id]



# Create cropped datasets with dynamic names
# Define the variable dynamically
data_variable <- get(paste0(data_to_forecast, "_data"))

# Create cropped datasets with dynamic names based on timescale and data_to_forecast
if (timescale == "forwards") {
  dependent_cropped <- data_variable[data_variable$agekaBP < (time_range[1]) & data_variable$agekaBP > time_range[2] + kyrs_to_forecast, ]
} else {
  dependent_cropped <- data_variable[data_variable$agekaBP < (time_range[2] - kyrs_to_forecast) & data_variable$agekaBP > time_range[1], ]
}

# Plot raw data

In [ ]:
# Plotting the raw data
###########################################
# Prepare data
if (timescale == "forwards") {
  dependent_toplot <- data_variable[data_variable$agekaBP < time_range[1] & data_variable$agekaBP > time_range[2], ]
} else {
  dependent_toplot <- data_variable[data_variable$agekaBP < time_range[2] & data_variable$agekaBP > time_range[1], ]
}

# List of cropped dataframes and their corresponding columns
cropped_data_list <- lapply(covariatenames, function(name) get(paste0(name, "_cropped")))

# Corresponding variable names
var_names <- covariatenames

# Create combined dataframe using a loop
combined_plot_data <- do.call(rbind, lapply(seq_along(cropped_data_list), function(i) {
  data.frame(agekaBP = cropped_data_list[[i]]$agekaBP,
             value = cropped_data_list[[i]][[var_names[i]]],
             variable = var_names[i])
}))

# Add the dependent variable
combined_plot_data <- rbind(
  combined_plot_data,
  data.frame(agekaBP = dependent_toplot$agekaBP, value = dependent_toplot[[data_to_forecast]], variable = data_to_forecast)
)

# Set the order of the variable levels
combined_plot_data$variable <- factor(combined_plot_data$variable, levels = c(var_names, data_to_forecast))

# Add a column to conditionally color the O18 plot
if (timescale == "forwards") {
  combined_plot_data$color <- ifelse(combined_plot_data$variable == data_to_forecast & combined_plot_data$agekaBP < (time_range[2] + kyrs_to_forecast), "red", "black")
} else {
  combined_plot_data$color <- ifelse(combined_plot_data$variable == data_to_forecast & combined_plot_data$agekaBP > (time_range[2] - kyrs_to_forecast), "red", "black")
}

# Separate the dependent data for custom styling
dependent_data <- combined_plot_data[combined_plot_data$variable == data_to_forecast & combined_plot_data$color == "red", ]

# Plotting data
if (timescale == "forwards") {
  # Plot with facet_wrap and custom color
  plotraw <- ggplot() +
    # Lines for all data excluding the red-colored dependent data
    geom_line(data = combined_plot_data[!(combined_plot_data$variable == data_to_forecast & combined_plot_data$color == "red"), ],
              aes(x = agekaBP, y = value, color = color)) +
    # Points for the red-colored dependent data
    geom_point(data = dependent_data, aes(x = agekaBP, y = value, color = color), size =0.1) +
    # Add a vertical line
    geom_vline(xintercept = (time_range[2] + kyrs_to_forecast), linetype = "dashed", color = "red") +
    # Facet wrap
    facet_wrap(~ variable, scales = "free_y", ncol = 2) +
    scale_y_continuous(n.breaks = 3)+
    scale_color_identity() +
    labs(x = "Age (ka BP)", y = "Value") +
    theme_bw()#+
    #theme(text = element_text(family = "LM Sans 10"))
} else {
  # Plot with facet_wrap and custom color
  plotraw <- ggplot() +
    # Lines for all data excluding the red-colored dependent data
    geom_line(data = combined_plot_data[!(combined_plot_data$variable == data_to_forecast & combined_plot_data$color == "red"), ],
              aes(x = agekaBP, y = value, color = color)) +
    # Points for the red-colored dependent data
    geom_point(data = dependent_data, aes(x = agekaBP, y = value, color = color), size =0.1) +
    # Add a vertical line
    geom_vline(xintercept = (time_range[2] - kyrs_to_forecast), linetype = "dashed", color = "red") +
    # Facet wrap
    facet_wrap(~ variable, scales = "free_y", ncol = 2) +
    scale_y_continuous(n.breaks = 3)+
    scale_color_identity() +
    labs(x = "Age (ka BP)", y = "Value") +
    theme_bw()#+
    #theme(text = element_text(family = "LM Sans 10"))
}

# Display the plot
print(plotraw)

# Prepare the data for Jags

In [ ]:
# Calculate means and standard deviations
mean_y <- tapply(combined_data$covariates_proxy_value, combined_data$covariates_proxy_id, mean)
sd_y <- tapply(combined_data$covariates_proxy_value, combined_data$covariates_proxy_id, sd)

# Standardize y
y_std <- numeric(length(combined_data$covariates_proxy_value))
for (i in 1:length(combined_data$covariates_proxy_value)) {
  y_std[i] <- (combined_data$covariates_proxy_value[i] - mean_y[covariates_proxy_id[i]]) / sd_y[covariates_proxy_id[i]]
}

# Precompute norm_prob for each covariate based on time_idx and age_error_sd
int_range <- 3
N_obs <- nrow(combined_data)
N <- length(new_time)
n_proxies <- length(unique(combined_data$covariates_proxy_id))

# Initialize the matrix with zeros
norm_prob_covariate <- matrix(0, nrow = N_obs, ncol = N)

# Loop over each observation to calculate and normalize probabilities within int_range
# if (timescale == "backwards") {
for (i in 1:N_obs) {
  # Define the subsetting range for each observation
  start_idx <- max(1, combined_data$covariates_time_idx[i] - int_range)
  end_idx <- min(N, combined_data$covariates_time_idx[i] + int_range)

  # Calculate probabilities within the subsetting range
  prob_slice <- dnorm(start_idx:end_idx, mean = combined_data$covariates_time_idx[i],
                      sd = combined_data$covariates_age_uncertainty[i])

  # Normalize the calculated probabilities
  prob_slice <- prob_slice / sum(prob_slice)

  # Assign normalized probabilities back to the sparse matrix within range
  norm_prob_covariate[i, start_idx:end_idx] <- prob_slice
}


# Define JAGS data for the covariates
jags_data_covariates <- list(
  int_range = 3,  # Pass int_range from R
  y_std = y_std,
  # y = combined_data$covariates_proxy_value,
  proxy_id = combined_data$covariates_proxy_id,
  time_idx = combined_data$covariates_time_idx,
  n_proxies = length(unique(combined_data$covariates_proxy_id)),
  N = length(new_time),
  N_obs = nrow(combined_data),
  age_error_sd = combined_data$covariates_age_uncertainty,
  proxy_error_sd = combined_data$covariates_proxy_uncertainty,
  mean_y = mean_y,
  age_ka_BP = combined_data$covariates_agekaBP,
  sd_y = sd_y,
  norm_prob_covariate = norm_prob_covariate
)

# Calculate means and standard deviations for each covariate
###############################
# List of proxy names
covariatenames <- var_names

# Initialize lists to store means and standard deviations
mean_list <- list()
sd_list <- list()

# Loop over each proxy
for (proxyname in covariatenames) {

  # Get the cropped data dynamically
  proxy_data <- get(paste0(proxyname, "_cropped"))

  # Calculate mean and standard deviation
  mean_value <- mean(proxy_data[[proxyname]], na.rm = TRUE)
  sd_value <- sd(proxy_data[[proxyname]], na.rm = TRUE)

  # Store the results in the lists
  mean_list[[proxyname]] <- mean_value
  sd_list[[proxyname]] <- sd_value
}


if (timescale == "forwards") {
  new_time_shrt_sorted <- sort(new_time_shrt)
  time_idx_shrt = findInterval(dependent_cropped$agekaBP, new_time_shrt_sorted)
  reversed_time_idx_shrt <- length(new_time_shrt_sorted) - time_idx_shrt + 1
  jags_data_dependent <- list(
    int_range = 3,  # Pass int_range from R
    y_shrt = dependent_cropped[[data_to_forecast]],
    N_obs_shrt = nrow(dependent_cropped),
    N_shrt = length(new_time_shrt),
    time_idx_shrt = reversed_time_idx_shrt,
    age_error_sd = dependent_cropped$uncertainty_1sigma,
    proxy_error_sd = dependent_cropped$measurement_uncertainty,
    age_ka_BP = dependent_cropped$median
  )
} else {
  jags_data_dependent <- list(
    int_range = 3,  # Pass int_range from R
    y_shrt = dependent_cropped[[data_to_forecast]],
    N_obs_shrt = nrow(dependent_cropped),
    N_shrt = length(new_time_shrt),
    time_idx_shrt = findInterval(dependent_cropped$agekaBP, new_time_shrt),
    age_error_sd = dependent_cropped$uncertainty_1sigma,
    proxy_error_sd = dependent_cropped$measurement_uncertainty,
    age_ka_BP = dependent_cropped$median
  )
}

# Initialize sparse matrix with zeros
norm_prob_dependent <- matrix(0, nrow = jags_data_dependent$N_obs_shrt, ncol = jags_data_dependent$N_shrt)

# Loop over each observation to calculate and normalize probabilities within int_range
for (i in 1:jags_data_dependent$N_obs_shrt) {
  # Define valid index range
  start_idx <- max(1, jags_data_dependent$time_idx_shrt[i] - int_range)
  end_idx <- min(jags_data_dependent$N_shrt, jags_data_dependent$time_idx_shrt[i] + int_range)

  # Compute probability slice only for the relevant range
  # time_range <- start_idx:end_idx
  prob_dependent <- dnorm(start_idx:end_idx, mean = jags_data_dependent$time_idx_shrt[i], sd = jags_data_dependent$age_error_sd[i])

  # Normalize and assign only to the sparse subset
  norm_prob_dependent[i, start_idx:end_idx] <- prob_dependent / sum(prob_dependent)
}


jags_data_dependent$norm_prob_dependent <- norm_prob_dependent

# Define the Jags function for KF of covarietes

In [ ]:
kf_covariates_jags <- function(jags_data,
                               #new_time,
                               #new_time_shrt,
                               n.chains = 3,
                               burnin = 1000,
                               adapt = 2000,
                               sample = 3000,
                               thin = 2,
                               initlist = NA,
                               model = "noerror" # "noerror" "eiv" "onlyageerror"
                               ) {

  kf_covariates_model <- "
    model {
  # Process noise and observation noise
  sigma_proc ~ dt(0, 1 / (1^2), 1) T(0, )
  sigma_obs ~ dt(0, 1 / (1^2), 1) T(0, )

        # Initial state
        x_std[1] ~ dnorm(0, 4^-2)

        for (t in 2:N) {
          x_std[t] ~ dnorm(x_std[t-1], 1 / sigma_proc^2)
        }

      for (i in 1:N_obs) {
        y_std[i] ~ dnorm(x_std[time_idx[i]], 1 / sigma_obs^2)
      }
      # Back transform
        for (t in 1:N) {
          x[t] <- x_std[t] * sd_y + mean_y
        }
    }
    "


kf_covariates_model_ageerror <- "
model {

  # Process noise and observation noise
  sigma_proc ~ dt(0, 1 / (1^2), 1) T(0, )
  sigma_obs ~ dt(0, 1 / (1^2), 1) T(0, )

  # Initial state
  x_std[1] ~ dnorm(0, 4^-2)

  for (t in 2:N) {
    x_std[t] ~ dnorm(x_std[t-1], 1 / sigma_proc^2)
  }

   for (i in 1:N_obs) {

    # Latent time index as a categorical variable using precomputed norm_prob
    true_time_idx[i] ~ dcat(norm_prob_covariate[i, 1:N])

    # Observation model with the latent time index
    y_std[i] ~ dnorm(x_std[true_time_idx[i]], 1 / (sigma_obs^2))
  }

  # Back transform
  for (t in 1:N) {
    x[t] <- x_std[t] * sd_y + mean_y
  }
}
"


  if (model == "noerror") {
    # Write the model to a file
    model <- "kalman_model.jags"
    writeLines(kf_covariates_model, con = model)
    parameters <- c("x", "sigma_proc", "sigma_obs")
  } else if (model == "onlyageerror"){
    # Write the model to a file
    model <- "kalman_model.jags"
    writeLines(kf_covariates_model_ageerror, con = model)
    parameters <- c("x", "sigma_proc", "sigma_obs")
  }



  # Run the JAGS model
  jags_covariates_kf_model <- run.jags(
    model = model,
    data = jags_data,
    monitor = parameters,
    n.chains = n.chains,
    burnin = burnin,
    sample = sample,
    adapt = adapt,
    thin = thin,
    method="parallel"
  )

  return(jags_covariates_kf_model)
}

# Run the KF for the covariates Jags model

In [ ]:
# To store results from each proxy
jags_model_result_covariates_list <- list()

In [ ]:
# Loop through each proxy
for (i in 1:length(covariatenames)) {#jags_data_covariates$n_proxies) { 1:length(covariatenames)

  # Filter data for the current proxy
   # Filter data for the current proxy
  jags_data_subset <- list(
    y_std = jags_data_covariates$y_std[jags_data_covariates$proxy_id == i],
    time_idx = jags_data_covariates$time_idx[jags_data_covariates$proxy_id == i],
    N_obs = sum(jags_data_covariates$proxy_id == i),
    N = jags_data_covariates$N,
    # age_error_sd = jags_data_covariates$age_error_sd[jags_data_covariates$proxy_id == i],
    # proxy_error_sd = jags_data_covariates$proxy_error_sd[jags_data_covariates$proxy_id == i],
    mean_y = as.numeric(jags_data_covariates$mean_y[i]),
    sd_y = as.numeric(jags_data_covariates$sd_y[i]),
    # int_range = 3,
    age_ka_BP = jags_data_covariates$age_ka_BP[jags_data_covariates$proxy_id == i],
    norm_prob_covariate = jags_data_covariates$norm_prob_covariate[jags_data_covariates$proxy_id == i,]
    # n_proxies = 1  # Only one proxy in the subset
  )

  # Compute the differences between consecutive ages
  age_spacings <- diff(sort(jags_data_covariates$age_ka_BP[jags_data_covariates$proxy_id == i]))
  # Select a quantile threshold (e.g., 25th percentile)
  criterion <- quantile(age_spacings, 0.5)


  # assign the model type according to the observation points of each proxy
  if (criterion > (0.5)) {
    kf_model <- "onlyageerror"
  } else {
    kf_model <- "noerror"
  }

  inits <- list(
  list(
    sigma_proc = 0.05,
    sigma_obs = 0.1,
    true_time = jags_data_subset$age_ka_BP
  ),
  list(
    sigma_proc = 0.2,
    sigma_obs = 0.05,
    true_time = jags_data_subset$age_ka_BP
  ),
  list(
    sigma_proc = 0.1,
    sigma_obs = 0.2,
    true_time = jags_data_subset$age_ka_BP
  )
)


  # Run the covariates model for this proxy
  jags_model_result_covariates <- kf_covariates_jags(
    jags_data = jags_data_subset,
    n.chains = 3,
    burnin = 1000,
    adapt = 500,
    sample = 2000,
    thin = 10,
    initlist = NA, #inits,  # Assuming you have the inits defined
    model = kf_model #"noerror"  # Modify as needed for different models: "noerror", "eiv" "fullstochastic" "onlyageerror"
  )

  # Store the results
  jags_model_result_covariates_list[[i]] <- jags_model_result_covariates

  # Optionally print progress
  print(paste("Completed proxy:", i))
}



# Extend results if needed

In [ ]:
# ---------- CHECK CONVERGENCE AND EXTEND IF NEEDED ----------
for (i in 1:length(covariatenames)) {
  model_result <- jags_model_result_covariates_list[[i]]

  # Convert to coda object for diagnostics
  coda_samples <- as.mcmc.list(model_result)

  # Compute Gelman-Rubin statistic
  gelman_result <- tryCatch(gelman.diag(coda_samples), error = function(e) NULL)

  if (!is.null(gelman_result)) {
    # Extract max potential scale reduction factor (PSRF)
    max_psrf <- max(gelman_result$psrf[, 1], na.rm = TRUE)

    if (max_psrf > 1.1) {  # Common threshold for non-convergence
      print(paste("Extending run for proxy", i, "due to poor convergence (max PSRF:", round(max_psrf, 3), ")"))

      # Extend the MCMC run
      extended_results <- extend.JAGS(model_result,
                                      burnin = 0,
                                      sample = 20)

      # Update stored results
      jags_model_result_covariates_list[[i]] <- extended_results

      print(paste("Extended run for proxy:", i))
    } else {
      print(paste("Proxy", i, "has converged (max PSRF:", round(max_psrf, 3), ")"))
    }
  } else {
    print(paste("Gelman diagnostic failed for proxy:", i))
  }
}


# Save image up to here

In [ ]:
nameworkspace <- paste("workspace_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".Rdata", sep = "")
save.image(file=paste("workspace_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".Rdata", sep = ""))


# Define the Jags function for KF of the dependent variable

In [ ]:
kf_dependent_jags <- function(jags_data,
                              n.chains = 3,
                              burnin = 1000,
                              adapt = 2000,
                              sample = 3000,
                              thin = 2,
                              initlist = NA,
                              model = "noerror" # "noerror" "eiv" "onlyageerror"
                              ) {

  kf_dependent_model <- "
    data {
      ym <- mean(y_shrt)
      ysd <- sd(y_shrt)
      for ( i in 1:N_obs_shrt ) {
        y_shrt_std[i] <- (y_shrt[i] - ym) / ysd
      }
    }
    model {
    # Process noise and observation noise
    sigma_proc_shrt ~ dt(0, 1 / (1^2), 1) T(0, )
    sigma_obs_shrt ~ dt(0, 1 / (1^2), 1) T(0, )

      # Initial state
      x_shrt_std[1] ~ dnorm(0, 4^-2)

      for (t in 2:N_shrt) {
        x_shrt_std[t] ~ dnorm(x_shrt_std[t-1], 1 / sigma_proc_shrt^2)
      }

      for (md in 1:N_obs_shrt){
        y_shrt_std[md] ~ dnorm(x_shrt_std[time_idx_shrt[md]], 1 / sigma_obs_shrt^2)
      }
      # Back transform
      for (t in 1:N_shrt) {
      x_shrt[t] <- x_shrt_std[t] * ysd + ym
  }
    }
  "


kf_dependent_model_ageerror <- "
data {
  ym <- mean(y_shrt)
  ysd <- sd(y_shrt)
  for (i in 1:N_obs_shrt) {
    y_shrt_std[i] <- (y_shrt[i] - ym) / ysd
  }
}

model {

  # Process noise and observation noise
  sigma_proc_shrt ~ dt(0, 1 / (1^2), 1) T(0, )
  sigma_obs_shrt ~ dt(0, 1 / (1^2), 1) T(0, )


  # Initial state
  x_shrt_std[1] ~ dnorm(0, 4^-2)  # Prior for initial state

  # State process with Gaussian random walk
  for (t in 2:N_shrt) {
    x_shrt_std[t] ~ dnorm(x_shrt_std[t-1], 1 / sigma_proc_shrt^2)
  }

  for (i in 1:N_obs_shrt) {
    # Latent time index as a categorical variable using precomputed norm_prob_dependent
    true_time_idx_shrt[i] ~ dcat(norm_prob_dependent[i, 1:N_shrt])

    # Observation model with the latent time index
    y_shrt_std[i] ~ dnorm(x_shrt_std[true_time_idx_shrt[i]], 1 / sigma_obs_shrt^2)
  }


  # Back-transform for output
  for (t in 1:N_shrt) {
    x_shrt[t] <- x_shrt_std[t] * ysd + ym
  }
}
"





  if (model == "noerror") {
    # Write the model to a file
    model <- "kalman_model_dependent.jags"
    writeLines(kf_dependent_model, con = model)
    parameters <- c("x_shrt", "sigma_proc_shrt", "sigma_obs_shrt")
  } else if (model == "onlyageerror"){
    # Write the model to a file
    model <- "kalman_model.jags"
    writeLines(kf_dependent_model_ageerror, con = model)
    parameters <- c("x_shrt", "sigma_proc_shrt", "sigma_obs_shrt", "sigma_age_shrt")
  }


  # Run the JAGS model
  jags_dependent_kf_model <- run.jags(
    model = model,
    data = jags_data,
    monitor = parameters,
    n.chains = n.chains,
    burnin = burnin,
    sample = sample,
    adapt = adapt,
    thin = thin,
    method = "parallel"
  )

  return(jags_dependent_kf_model)
}

# Run the KF for dependent Jags model

In [ ]:
# Run the dependent model
jags_model_result_dependent <- kf_dependent_jags(
  jags_data = jags_data_dependent,
  n.chains = 3,
  burnin = 2000,
  adapt = 1000,
  sample = 4000,
  thin = 2,
  #initlist = inits_dependent,
  model = "onlyageerror" # "noerror" "eiv" "onlyageerror" "fullstochastic"
)

In [ ]:

  # Convert to coda object for diagnostics
  coda_samples_dependent <- as.mcmc.list(jags_model_result_dependent)

  # Compute Gelman-Rubin statistic
  gelman_result_dependent <- tryCatch(gelman.diag(coda_samples), error = function(e) NULL)

  if (!is.null(gelman_result_dependent)) {
    # Extract max potential scale reduction factor (PSRF)
    max_psrf <- max(gelman_result_dependent$psrf[, 1], na.rm = TRUE)

    if (max_psrf > 1.1) {  # Common threshold for non-convergence
      print(paste("Extending run for proxy due to poor convergence (max PSRF:", round(max_psrf, 3), ")"))

      # Extend the MCMC run
      extended_results <- extend.JAGS(jags_model_result_dependent,
                                      burnin = 0,
                                      sample = 2000)

      # Update stored results
      jags_model_result_dependent <- extended_results

      print(paste("Extended run for proxy"))
    } else {
      print(paste("Proxy has converged (max PSRF:", round(max_psrf, 3), ")"))
    }
  } else {
    print(paste("Gelman diagnostic failed for proxy:", i))
  }


[1] "Gelman diagnostic failed for proxy: 13"


# Trace plots of KF runs

In [ ]:
mcmc_trace_covariates_pars_kf <- c("sigma_proc", "sigma_obs")

mcmc_trace_dependent_pars_kf <- c("sigma_proc_shrt", "sigma_obs_shrt")

In [ ]:
mcmc_trace_list <- lapply(1:length(jags_model_result_covariates_list), function(i) {
  mcmc_trace(jags_model_result_covariates_list[[i]]$mcmc, pars = mcmc_trace_covariates_pars_kf) +
    ggtitle(paste("KF trace plots - ", covariatenames[i])) +  # Use covariate names in the title
    theme(plot.title = element_text(size = 10, hjust = .5)) +
    theme(legend.position = "none",
          axis.text = element_text(size = 10),
          axis.title = element_text(size = 11)) +
    theme(axis.title.x = element_blank(),
          axis.text.x = element_blank(), axis.ticks.x=element_blank(), axis.ticks.y=element_blank(), axis.text.y= element_blank(),
          strip.text = element_text(size = 5)
          #, text=element_text(family="LM Sans 10")
          )
})


# Create a single plot by combining all plots in mcmc_trace_list using patchwork
mcmc_trace_covariates_kf <- wrap_plots(mcmc_trace_list, ncol = 3) +  # Adjust ncol for desired layout
  plot_annotation(title = paste("KF trace plots ", time_range[1],"-", time_range[2]),
                  theme = theme(plot.title = element_text(size = 25, hjust = .5) #,family="LM Sans 10")
                  )

# Display the combined plot
# print(mcmc_trace_covariates_kf)


mcmc_trace_dependent_kf <- mcmc_trace(jags_model_result_dependent$mcmc, pars = mcmc_trace_dependent_pars_kf) +
  ggtitle(paste("KF trace plots - ", data_to_forecast)) +
  #theme(text=element_text(family="LM Sans 10")) +
  theme(plot.title = element_text(size = 10, hjust = .5)) +
    theme(legend.position = "none",
          axis.text = element_text(size = 10),
          axis.title = element_text(size = 11)) +
    theme(axis.title.x = element_blank(),
          axis.text.x = element_blank(), axis.ticks.x=element_blank(), axis.ticks.y=element_blank(), axis.text.y= element_blank(),
          strip.text = element_text(size = 5)#, text=element_text(family="LM Sans 10")
          )


# print(mcmc_trace_dependent_kf)

# Append the dependent trace plot to the list of covariate trace plots
mcmc_trace_list <- c(mcmc_trace_list, list(mcmc_trace_dependent_kf))

# Combine all plots into a single grid layout using patchwork
mcmc_trace_combined_kf <- wrap_plots(mcmc_trace_list, ncol = 3) +  # Adjust `ncol` for layout
  plot_annotation(title = paste("KF trace plots ", time_range[1],"-", time_range[2]),
                  theme = theme(plot.title = element_text(size = 25, hjust = .5))) #, family = "LM Sans 10"

# Display the combined plot
print(mcmc_trace_combined_kf)

In [ ]:
# Set seed for reproducibility when sampling x indices
set.seed(123)

# Get the number of `x` elements for both covariates and dependent models
n_x_covariates <- dim(jags_model_result_covariates_list[[1]]$mcmc[[1]])[2]  # For covariates model
n_x_dependent <- dim(jags_model_result_dependent$mcmc[[1]])[2]  # For dependent model

# Randomly select 3 indices for x from each model
random_x_indices_covariates <- sample(1:n_x_covariates, 3)
random_x_indices_dependent <- sample(1:n_x_dependent, 3)

# Define parameter names for the randomly selected x indices
mcmc_trace_random_x_covariates <- paste0("x[", random_x_indices_covariates, "]")
mcmc_trace_random_x_dependent <- paste0("x_shrt[", random_x_indices_dependent, "]")  # Use `x_shrt` for dependent model

# Plot MCMC traces for the selected x indices for covariates model
mcmc_trace_random_x_covariates_plots <- lapply(1:length(jags_model_result_covariates_list), function(i) {
  mcmc_trace(jags_model_result_covariates_list[[i]]$mcmc, pars = mcmc_trace_random_x_covariates) +
    ggtitle(paste("KF trace - Random indices (", covariatenames[i], ")")) +
    theme(plot.title = element_text(size = 10, hjust = .5)) +
    theme(legend.position = "none",
          axis.text = element_text(size = 10),
          axis.title = element_text(size = 11)) +
    theme(axis.title.x = element_blank(),
          axis.text.x = element_blank(), axis.ticks.x=element_blank(), axis.ticks.y=element_blank(), axis.text.y= element_blank(),
          strip.text = element_text(size = 5)) #, text=element_text(family="LM Sans 10")
})

# Combine covariates' plots into one layout
mcmc_trace_random_x_covariates_combined <- wrap_plots(mcmc_trace_random_x_covariates_plots, ncol = 3) +
  plot_annotation(title = paste("KF trace plots for random indices", time_range[1],"-", time_range[2]),
                  theme = theme(plot.title = element_text(size = 15, hjust = .5))) #, family="LM Sans 10"

# Plot MCMC traces for the selected x_shrt indices for dependent model
mcmc_trace_random_x_dependent_plot <- mcmc_trace(jags_model_result_dependent$mcmc, pars = mcmc_trace_random_x_dependent) +
  ggtitle(paste("KF trace - Random indices (", data_to_forecast, ")")) +
  theme(plot.title = element_text(size = 10, hjust = .5)) +
  theme(legend.position = "none",
        axis.text = element_text(size = 10),
        axis.title = element_text(size = 11)) +
  theme(axis.title.x = element_blank(),
        axis.text.x = element_blank(), axis.ticks.x=element_blank(), axis.ticks.y=element_blank(), axis.text.y= element_blank(),
        strip.text = element_text(size = 5)) #, text=element_text(family="LM Sans 10")

# Combine both covariates and dependent model plots
combined_trace_plots_random_x <- mcmc_trace_random_x_covariates_combined + mcmc_trace_random_x_dependent_plot

# Display the combined plot
print(combined_trace_plots_random_x)


# Extract the KF results

In [ ]:
# Extract the results
results_covariates_kf <- as.mcmc(jags_model_result_covariates)
x_samples_covariates_kf <- as.matrix(results_covariates_kf)

# Extract the results
results_dependent_kf <- as.mcmc(jags_model_result_dependent)
x_samples_dependent_kf <- as.matrix(results_dependent_kf)

# Define proxies and index prefixes
proxy_names <- c(var_names, data_to_forecast)
proxy_length <- 1:length(proxy_names)  # Assuming proxy indices follow this order

# Initialize an empty list to store the data frames
proxy_dfs <- list()

# Loop through proxies and process
for (i in proxy_length) {
  if (i <= (length(proxy_names) - 1)) {
    # Extract the MCMC list for the i-th proxy
    mcmc_samples <- jags_model_result_covariates_list[[i]]$mcmc

    # Combine MCMC samples across chains (if necessary)
    combined_samples <- do.call(rbind, mcmc_samples)

    # Filter columns that start with "x" (e.g., x[1], x[2], ...)
    x_columns <- grep("^x\\[", colnames(combined_samples), value = TRUE)
    filtered_samples <- combined_samples[, x_columns]

    # Initialize a matrix to store the summary stats for each time point
    n_time_points <- ncol(filtered_samples)
    summary_stats <- matrix(NA, nrow = n_time_points, ncol = 4)
    colnames(summary_stats) <- c("median_kf", "lower_kf", "upper_kf", "sd_kf")

    # Loop through each time series (x[1], x[2], ...)
    for (j in 1:n_time_points) {
      # Compute the median, 2.5%, and 97.5% quantiles for the j-th time series
      summary_stats[j, "median_kf"] <- median(filtered_samples[, j])
      summary_stats[j, "lower_kf"] <- quantile(filtered_samples[, j], 0.025)
      summary_stats[j, "upper_kf"] <- quantile(filtered_samples[, j], 0.975)
      summary_stats[j, "sd_kf"] <- sd(filtered_samples[, j])
    }

    # Store the summary stats for the i-th proxy
    proxy_dfs[[paste0("Proxy_", i)]] <- summary_stats
  } else {
    # For dependent (different source, x_samples_dependent_kf)
    proxy_indices <- grep("^x_shrt", colnames(x_samples_dependent_kf))
    x_proxy <- apply(x_samples_dependent_kf[, proxy_indices], 2, median)
    x_proxy_lower <- apply(x_samples_dependent_kf[, proxy_indices], 2, quantile, 0.025)
    x_proxy_upper <- apply(x_samples_dependent_kf[, proxy_indices], 2, quantile, 0.975)
    x_proxy_sd <- apply(x_samples_dependent_kf[, proxy_indices], 2, sd)

    # Create a dataframe and store it
    proxy_df <- data.frame(proxy_indices, x_proxy, x_proxy_lower, x_proxy_upper, x_proxy_sd)
    proxy_dfs[[data_to_forecast]] <- proxy_df
  }
}


# Extend dependent to match the length of new_time by padding with NAs
dependent_df <- proxy_dfs[[data_to_forecast]]  # Dynamically select based on data_to_forecast
dependent_extended_df <- data.frame(
  agekaBP_kf = new_time,  # Use the full length of new_time
  median_kf = c(dependent_df[, 2], rep(NA, length(new_time) - length(new_time_shrt))),
  upper_kf = c(dependent_df[, 4], rep(NA, length(new_time) - length(new_time_shrt))),
  lower_kf = c(dependent_df[, 3], rep(NA, length(new_time) - length(new_time_shrt))),
  sd_kf = c(dependent_df[, 5], rep(NA, length(new_time) - length(new_time_shrt)))
)

# Replace the original O18 or C13 data frame with the extended version
proxy_dfs[[data_to_forecast]] <- dependent_extended_df

# Combine data frames into one for plotting
combined_plot_kf <- do.call(rbind, lapply(seq_along(proxy_names), function(i) {
  # Handle proxies stored as matrices (Proxy_1, Proxy_2, etc.)
  if (i <= (length(proxy_names) - 1)) {
    proxy_df <- proxy_dfs[[paste0("Proxy_", i)]]

    # Create a data frame for the current proxy with relevant columns
    data.frame(agekaBP_kf = new_time,  # Assuming 'new_time' is a vector with time points
               value_kf = proxy_df[, "median_kf"],  # Median values
               upper_kf = proxy_df[, "upper_kf"],  # Upper bound (97.5% quantile)
               lower_kf = proxy_df[, "lower_kf"],  # Lower bound (2.5% quantile)
               #lower_kf = proxy_df[, "sd_kf"],  # sd
               variable = proxy_names[i])  # Proxy name for identification
  } else {
    # Handle O18 separately since it's stored as a data frame
    dependent_plot_df <- proxy_dfs[[data_to_forecast]]

    # Create a data frame for O18
    data.frame(agekaBP_kf = dependent_plot_df$agekaBP_kf,
               value_kf = dependent_plot_df$median_kf,
               upper_kf = dependent_plot_df$upper_kf,
               lower_kf = dependent_plot_df$lower_kf,
               #sd_kf = o18_df$sd_kf,
               variable = data_to_forecast)  # Name O18 explicitly
  }
}))


# Dynamically adjust the factor levels based on the data_to_forecast
combined_plot_kf$variable <- factor(combined_plot_kf$variable,
                                    levels = c(var_names, data_to_forecast))

# Assuming 'O18' or 'C13' is the last dataset in combined_plot_data
combined_plot_data <- combined_plot_data %>%
  mutate(color = ifelse(variable == data_to_forecast & agekaBP > (time_range[2] - kyrs_to_forecast), "red", "black"))

# Plot the result

In [ ]:
# Add a column to conditionally color the dependent data plot
# Plotting data
if (timescale == "forwards") {
  combined_plot_data$color <- ifelse(combined_plot_data$variable == data_to_forecast & combined_plot_data$agekaBP < (time_range[2] + kyrs_to_forecast), "red", "black")
} else {
  combined_plot_data$color <- ifelse(combined_plot_data$variable == data_to_forecast & combined_plot_data$agekaBP > (time_range[2] - kyrs_to_forecast), "red", "black")
}

#options(warn=-1)

if (timescale == "forwards") {
  # Plot with facet_wrap and custom color
  plot_result_kf <- ggplot(combined_plot_kf, aes(x = agekaBP_kf, y = value_kf)) +
  geom_point(data = combined_plot_data, aes(x = agekaBP, y = value, color = color), size = 0.3) +
  geom_line(aes(color = "Filtered"), linewidth = 0.5) +
  geom_vline(xintercept = (time_range[2] + kyrs_to_forecast), linetype = "dashed", color = "red") +
  facet_wrap(~ variable, scales = "free_y", ncol = 2) +
  geom_ribbon(aes(ymin = lower_kf, ymax = upper_kf), fill = "cyan", alpha = 0.3, color = NA) +
  scale_color_manual(values = c("black", "blue", "red"),
                     labels = c("Data" = "Data", "Filtered" = "KF data")) +
  scale_y_continuous(n.breaks = 3)+
  labs(x = "Age (ka BP)", y = "Value") +
  theme_bw() +
  theme(
    #text = element_text(family = "LM Sans 10"),
    legend.position = "none",
    strip.placement = "outside",
    strip.text.y = element_text(angle = 0, hjust = 0, margin = margin(r = 10)),
    axis.text.y.right = element_text(margin = margin(l = 10))
  ) +
  scale_fill_identity()
} else {
  # Plot with facet_wrap and custom color
plot_result_kf <- ggplot(combined_plot_kf, aes(x = agekaBP_kf, y = value_kf)) +
  geom_point(data = combined_plot_data, aes(x = agekaBP, y = value, color = color), size = 0.3) +
  geom_line(aes(color = "Filtered"), linewidth = 0.5) +
  geom_vline(xintercept = (time_range[2] - kyrs_to_forecast), linetype = "dashed", color = "red") +
  facet_wrap(~ variable, scales = "free_y", ncol = 2) +
  geom_ribbon(aes(ymin = lower_kf, ymax = upper_kf), fill = "cyan", alpha = 0.3, color = NA) +
  scale_color_manual(values = c("black", "blue", "red"),
                     labels = c("Data" = "Data", "Filtered" = "KF data")) +
  scale_y_continuous(n.breaks = 3)+
  labs(x = "Age (ka BP)", y = "Value") +
  theme_bw() +
  theme(
    #text = element_text(family = "LM Sans 10"),
    legend.position = "none",
    strip.placement = "outside",
    strip.text.y = element_text(angle = 0, hjust = 0, margin = margin(r = 10)),
    axis.text.y.right = element_text(margin = margin(l = 10))
  ) +
  scale_fill_identity()
}

# Print the plot
print(plot_result_kf)

# Prepare data for the Jags regression model

In [ ]:
# Change the names of proxies
names(proxy_dfs)[1:length(var_names)] <- var_names

# Create lists for median and sd values
X <- list()
X_sd <- list()
for (var in var_names) {
  X[[paste0(var, "_median_kf")]] <- proxy_dfs[[var]][, "median_kf"]
  X_sd[[paste0(var, "_sd_kf")]] <- proxy_dfs[[var]][, "sd_kf"]
}

# Convert to data frames
X_df <- as.data.frame(X)
X_sd_df <- as.data.frame(X_sd)

# Store original Y and Y_sd
Y_original <- c(proxy_dfs[[data_to_forecast]][["median_kf"]])
Y_sd_original <- c(proxy_dfs[[data_to_forecast]][["sd_kf"]])

# Standardize X and Y
X_mean <- colMeans(X_df, na.rm = TRUE)
X_sd_val <- apply(X_df, 2, sd, na.rm = TRUE)
Y_mean <- mean(Y_original, na.rm = TRUE)
Y_sd_val <- sd(Y_original, na.rm = TRUE)

# Create standardized versions
X_std <- scale(X_df)
X_sd_std <- sweep(X_sd_df, 2, X_sd_val, "/")
Y_std <- (Y_original - Y_mean) / Y_sd_val
Y_sd_std <- Y_sd_original / Y_sd_val

# Prepare data for JAGS with standardized values
data_list_ssvs <- list(
  N = length(Y_std),
  P = ncol(X_std),
  Y = Y_std,
  Y_sd = Y_sd_std,
  X = as.matrix(X_std),
  X_sd = as.matrix(X_sd_std),
  n_predict = sum(is.na(Y_original))
)

# Initial values
inits_regression <- function() {
  list(beta = rnorm(P), gamma = rbinom(P, 1, 0.5), sigma_ss = runif(1, 0, 10), sigma_y = runif(1, 0, 10), sigma_x = runif(1, 0, 10))
}

# Define the Jags function for regression

In [ ]:
ssvs_jags <- function(jags_data,
                              n.chains = 3,
                              burnin = 1000,
                              adapt = 2000,
                              sample = 3000,
                              thin = 2,
                              initlist = NA,
                              model = c("simplyssvs", "rwlevel", "staticlevel", "ar1")) {

  # SSVS regression model
  regression_model_with_simplyssvs <- "
model {
    for (t in 1:(N-n_predict)) {
        Y[t] ~ dnorm(mu[t], tau_y[t])
        mu[t] <- inprod(X[t,], beta)
        tau_y[t] <- 1/((Y_sd[t] + sigma_y)^2) # 1/(Y_sd[t]^2 + sigma_y^2)

        for (j in 1:P) {
            X[t,j] ~ dnorm(state[t,j], tau_x[t,j])
            state[t,j] ~ dnorm(0, 0.01)  # More informative prior
            tau_x[t,j] <- 1/((X_sd[t,j] + sigma_x)^2) # 1/(X_sd[t,j]^2 + sigma_x^2)
        }
    }

    # Prediction part remains similar but with modified precisions
    for (t in (N-n_predict+1):N) {
        Y[t] ~ dnorm(mu[t], 1/sigma_y^2)
        mu[t] <- inprod(X[t,], beta)
        loglik[t] <- logdensity.norm(Y[t], mu[t], 1/sigma_y^2)

        for (j in 1:P) {
            X[t,j] ~ dnorm(state[t,j], tau_x[t,j])
            state[t,j] ~ dnorm(0, 0.1)
            tau_x[t,j] <- 1/(X_sd[t,j] + sigma_x)^2
        }
    }

    for (j in 1:P) {
        beta[j] <- gamma[j]*delta[j]
        gamma[j] ~ dbern(prob[j])
        prob[j] ~ dbeta(1, 1)
        delta[j] ~ dt(0, tau_ss, nu[j])
        nu[j] ~ dexp(1/30)  # Modified rate
    }

    # Modified variance priors
    tau_ss <- 1/sigma_ss^2
    sigma_ss ~ dt(0,4^-2,1)T(0,) # dgamma(1, 0.1) #dgamma(2, 1)
    sigma_y ~ dt(0,4^-2,1)T(0,) # dgamma(1, 0.1) #dgamma(2, 1)
    sigma_x ~ dt(0,4^-2,1)T(0,) # dgamma(1, 0.1) #dgamma(2, 1)

    # PPC
    for (t in 1:(N-n_predict)) {
        Y_rep[t] ~ dnorm(mu[t], tau_y[t])
    }
}
"


#   # SSVS regression model
#   regression_model_with_rw_level <- "
# model {
#     for (t in 1:(N-n_predict)) {
#         Y[t] ~ dnorm(mu[t], tau_y[t])
#         mu[t] <- level[t] + inprod(X[t,], beta)
#         tau_y[t] <- (Y_sd[t]^2 + (sigma_y)^2)^(-1)
#         for (j in 1:P) {
#           X[t,j] ~ dnorm(state[t,j], tau_x[t,j])
#           state[t,j] ~ dnorm(0, 0.001)
#           tau_x[t,j] <- (X_sd[t,j]^2 + (sigma_x)^2)^(-1)
#         }
#     }
#     for (t in (N-n_predict+1):N) {
#         Y[t] ~ dnorm(mu[t], sigma_y^-2)
#         mu[t] <- level[t] + inprod(X[t,], beta)
#         loglik[t] <- logdensity.norm(Y[t], mu[t], sigma_y^-2)
#         for (j in 1:P) {
#           X[t,j] ~ dnorm(state[t,j], tau_x[t,j])
#           state[t,j] ~ dnorm(0, 0.001)
#           tau_x[t,j] <- (X_sd[t,j]^2 + (sigma_x)^2)^(-1)
#         }
#     }
#     for (j in 1:P) {
#         beta[j] <- gamma[j]*delta[j]
#         # gamma[j] ~ dbern(0.5)
#         gamma[j] ~ dbern(prob[j])
#         prob[j] ~ dbeta(1, 1)  # Uniform prior for prob
#         delta[j] ~ dnorm(0, tau_ss)
#     }
#     level[1] ~ dnorm(0, 0.001)
#     for (t in 2:N) {
#         level[t] ~ dnorm(level[t-1], tau_lvl)
#     }
#     tau_ss <- pow(sigma_ss, -2)
#     tau_lvl <- pow(sigma_lvl, -2)
#     sigma_lvl ~ dt(0,4^-2,1)T(0,)
#     sigma_ss ~ dnorm(0,10^-2)T(0,)
#     sigma_y ~ dt(0,4^-2,1)T(0,)
#     sigma_x ~ dt(0,4^-2,1)T(0,)
# }
# "


# regression_model_with_ar1 <- "
# model {
#     for (t in 2:(N-n_predict)) {
#         Y[t] ~ dnorm(mu[t], tau_y[t])
#         mu[t] <- phi * Y[t-1] + inprod(X[t,], beta)
#         tau_y[t] <- (Y_sd[t]^2 + (sigma_y)^2)^(-1)
#         #loglik[t] <- logdensity.norm(Y[t], mu[t], tau_y[t])
#         for (j in 1:P) {
#           X[t,j] ~ dnorm(state[t,j], tau_x[t,j])
#           state[t,j] ~ dnorm(0, 0.001)
#           tau_x[t,j] <- (X_sd[t,j]^2 + (sigma_x)^2)^(-1)
#         }
#     }
#     for (t in (N-n_predict+1):N) {
#         Y[t] ~ dnorm(mu[t], sigma_y^-2)
#         mu[t] <- phi * Y[t-1] + inprod(X[t,], beta)
#         loglik[t] <- logdensity.norm(Y[t], mu[t], sigma_y^-2)
#         for (j in 1:P) {
#           X[t,j] ~ dnorm(state[t,j], tau_x[t,j])
#           state[t,j] ~ dnorm(0, 0.001)
#           tau_x[t,j] <- (X_sd[t,j]^2 + (sigma_x)^2)^(-1)
#         }
#     }
#     phi ~ dnorm(0, 0.001)
#     Y[1]~dnorm(0, 0.01)

#     for (j in 1:P) {
#         beta[j] <- gamma[j]*delta[j]
#         # gamma[j] ~ dbern(0.5)
#         gamma[j] ~ dbern(prob[j])
#         prob[j] ~ dbeta(1, 1)  # Uniform prior for prob
#         delta[j] ~ dnorm(0, tau_ss)
#     }

#     tau_ss <- pow(sigma_ss, -2)
#     sigma_ss ~ dnorm(0,10^-2)T(0,)
#     sigma_y ~ dt(0,4^-2,1)T(0,)
#     sigma_x ~ dt(0,4^-2,1)T(0,)
# }
# "

# # SSVS regression model
# regression_model_with_static_level <- "
# model {
#     for (t in 1:(N-n_predict)) {
#         Y[t] ~ dnorm(mu[t], tau_y[t])
#         mu[t] <- level[t] + inprod(X[t,], beta)
#         tau_y[t] <- (Y_sd[t]^2 + (sigma_y)^2)^(-1)
#         for (j in 1:P) {
#           X[t,j] ~ dnorm(state[t,j], tau_x[t,j])
#           state[t,j] ~ dnorm(0, 0.001)
#           tau_x[t,j] <- (X_sd[t,j]^2 + (sigma_x)^2)^(-1)
#         }
#     }
#     for (t in (N-n_predict+1):N) {
#         Y[t] ~ dnorm(mu[t], sigma_y^-2)
#         mu[t] <- level[t] + inprod(X[t,], beta)
#         loglik[t] <- logdensity.norm(Y[t], mu[t], sigma_y^-2)
#         for (j in 1:P) {
#           X[t,j] ~ dnorm(state[t,j], tau_x[t,j])
#           state[t,j] ~ dnorm(0, 0.001)
#           tau_x[t,j] <- (X_sd[t,j]^2 + (sigma_x)^2)^(-1)
#         }
#     }
#     for (j in 1:P) {
#         beta[j] <- gamma[j]*delta[j]
#         # gamma[j] ~ dbern(0.5)
#         gamma[j] ~ dbern(prob[j])
#         prob[j] ~ dbeta(1, 1)  # Uniform prior for prob
#         delta[j] ~ dnorm(0, tau_ss)
#     }
#     for (t in 1:N) {
#         level[t] ~ dnorm(0, tau_lvl)
#     }
#     tau_ss <- pow(sigma_ss, -2)
#     tau_lvl <- pow(sigma_lvl, -2)
#     sigma_lvl ~ dt(0,4^-2,1)T(0,)
#     sigma_ss ~ dnorm(0,10^-2)T(0,)
#     sigma_y ~ dt(0,4^-2,1)T(0,)
#     sigma_x ~ dt(0,4^-2,1)T(0,)
# }
# "


  if (model == "rwlevel") {
    # Write the model to a file
    model <- "regression_model_with_rw_level.jags"
    writeLines(regression_model_with_rw_level, con = model)
    parameters <- c("beta", "gamma", "delta", "sigma_ss", "sigma_y", "sigma_lvl", "level", "loglik", "sigma_x", "prob", "Y_rep")
  } else if (model == "simplyssvs") {
    # Write the model to a file
    model <- "regression_model_with_simplyssvs.jags"
    writeLines(regression_model_with_simplyssvs, con = model)
    parameters <- c("beta", "gamma", "delta", "sigma_ss", "sigma_y", "loglik", "sigma_x", "prob", "Y_rep")
  } else if (model == "ar1") {
    # Write the model to a file
    model <- "regression_model_with_ar1.jags"
    writeLines(regression_model_with_ar1, con = model)
    parameters <- c("beta", "gamma", "delta", "sigma_ss", "sigma_y", "phi", "loglik", "sigma_x", "prob")
  } else if (model == "staticlevel") {
    # Write the model to a file
    model <- "regression_model_with_static_level.jags"
    writeLines(regression_model_with_static_level, con = model)
    parameters <- c("beta", "gamma", "delta", "sigma_ss", "sigma_y", "sigma_lvl", "level", "loglik", "sigma_x", "prob")
  }


  # Parameters to monitor
  # parameters <- c("x_shrt", "sigma_proc_shrt", "sigma_obs_shrt", "sigma_age_shrt")

  # Run the JAGS model
  jags_regression_model <- run.jags(
    model = model,
    data = jags_data,
    monitor = parameters,
    n.chains = n.chains,
    burnin = burnin,
    sample = sample,
    adapt = adapt,
    thin = thin,
    #inits = initlist,
    method = "parallel",
    modules = c("glm", "dic")
  )

  return(jags_regression_model)
}


# Run the Jags regression model

In [ ]:
# Run the covariates model
jags_regression_model <- ssvs_jags(
  jags_data = data_list_ssvs,
  n.chains = 3,
  burnin = 5000,
  adapt = 5000,
  sample = 10000,
  thin = 2,
  initlist = inits_regression,
  model = mymodel
)

# In case it did not converge, to extend the runs

In [ ]:
  # Convert to coda object for diagnostics
  coda_samples_regression <- as.mcmc.list(jags_regression_model)

  # Compute Gelman-Rubin statistic
  gelman_result_regression <- tryCatch(gelman.diag(coda_samples_regression), error = function(e) NULL)

  if (!is.null(gelman_result_dependent)) {
    # Extract max potential scale reduction factor (PSRF)
    max_psrf <- max(gelman_result_regression$psrf[, 1], na.rm = TRUE)

    if (max_psrf > 1.1) {  # Common threshold for non-convergence
      print(paste("Extending run for proxy due to poor convergence (max PSRF:", round(max_psrf, 3), ")"))

      # Extend the MCMC run
      extended_results <- extend.JAGS(jags_regression_model,
                                      burnin = 0,
                                      sample = 20000)

      # Update stored results
      jags_regression_model <- extended_results

      print(paste("Extended run for regression"))
    } else {
      print(paste("Proxy has converged (max PSRF:", round(max_psrf, 3), ")"))
    }
  } else {
    print(paste("Gelman diagnostic failed for regression"))
  }

# Extract posterior samples

In [ ]:
# Extract and process posterior samples
samples <- as.mcmc.list(jags_regression_model)
beta_samples <- as.matrix(samples[, grep("beta", varnames(samples))])
loglik_samples <- as.matrix(samples[, grep("loglik", varnames(samples))])

# Generate predictions in standardized scale
X_matrix_std <- as.matrix(X_std)
Y_pred_std_samples <- beta_samples %*% t(X_matrix_std)

# Back-transform predictions to original scale
Y_pred_samples <- sweep(Y_pred_std_samples, 2, Y_sd_val, "*")
Y_pred_samples <- sweep(Y_pred_samples, 2, Y_mean, "+")

# Back-transform beta coefficients to original scale
beta_original_scale <- sweep(beta_samples, 2, Y_sd_val/X_sd_val, "*")

# Create summary dataframes
df_beta <- data.frame(
  Predictor = names(X),
  Mean = colMeans(beta_original_scale),
  CI_Lower = apply(beta_original_scale, 2, quantile, probs = 0.055),
  CI_Upper = apply(beta_original_scale, 2, quantile, probs = 0.945)
)

# Calculate prediction summaries
Y_pred_summary <- data.frame(
  Mean = apply(Y_pred_samples, 2, mean),
  Median = apply(Y_pred_samples, 2, median),
  SD = apply(Y_pred_samples, 2, sd),
  CI_Lower = apply(Y_pred_samples, 2, quantile, probs = 0.055),
  CI_Upper = apply(Y_pred_samples, 2, quantile, probs = 0.945)
)

  # Plot beta coefficients
  plot_beta <- ggplot(df_beta, aes(x = Predictor)) +
    geom_point(aes(y = Mean), color = "red") +
    geom_errorbar(aes(ymin = CI_Lower, ymax = CI_Upper), width = 0.2, color = "red") +
    labs(title = "Back-transformed Beta Coefficients",
         x = "Predictor",
         y = "Beta") +
    theme_minimal()

  # Save the beta plot
  ggsave(plot_beta,
         filename = "beta_coefficients_backtransformed.pdf",
         device = "pdf",
         height = 10, width = 10, units = "in")

  # Print the beta plot
  print(plot_beta)

# Plotting the regression results

In [ ]:
# Calculate summary statistics
Y_pred_median <- apply(Y_pred_samples, 2, median)
Y_pred_sd <- apply(Y_pred_samples, 2, sd)
Y_pred_ci <- apply(Y_pred_samples, 2, quantile, probs = c(0.005, 0.975))

# Print summary statistics for Y_pred
Y_pred_summary <- data.frame(
  Mean = Y_pred_median,
  SD = Y_pred_sd,
  CI_Lower = Y_pred_ci[1, ],
  CI_Upper = Y_pred_ci[2, ]
)

# Cropping O18 data to the specified date range
# Create cropped datasets with dynamic names
if (timescale == "forwards") {
  dependent_ssvs_cropped <- data_variable[data_variable$agekaBP < (time_range[1]) & data_variable$agekaBP > time_range[2], ]
} else {
  dependent_ssvs_cropped <- data_variable[data_variable$agekaBP < time_range[2] & data_variable$agekaBP > time_range[1], ]
}


plot_data <- list(
  predicteddata=data.frame(
    Time = 1:N,
    TimekaBP = proxy_dfs[[data_to_forecast]][["agekaBP_kf"]],
    Observed = proxy_dfs[[data_to_forecast]][["median_kf"]],
    Predicted = Y_pred_median,
    Lower = Y_pred_ci[1, ],
    Upper = Y_pred_ci[2, ]),
  observeddata=data.frame(
    data_to_predict = dependent_ssvs_cropped[[data_to_forecast]],
    age_data_to_predict = dependent_ssvs_cropped$agekaBP
  )
)


if (timescale == "forwards") {
  ssvs_result <- ggplot(plot_data$predicteddata, aes(x = Time)) +
    theme(legend.position = "right") +  # Removing the plot.title element
    geom_point(data=plot_data$observeddata, aes(x = age_data_to_predict, y = data_to_predict, color = "Data")) + # , linewidth = 0.5
    #geom_line(aes(x = seq(time_range[1],(time_range[2]), by=0.1), y = Observed, color = "Observed"), linewidth = 1) +
    geom_line(aes(x = seq(time_range[1],(time_range[2]), by=-resolution), y = Predicted, color = "Predicted"), linewidth = 0.75) +
    geom_ribbon(aes(x = seq(time_range[1],(time_range[2]), by=-resolution), ymin = Lower, ymax = Upper, fill = "Prediction Interval"), alpha = 0.5) +
    geom_point(data = plot_data$observeddata, aes(x = age_data_to_predict, y = data_to_predict, color = "Data"), alpha = 1, size = 1) +
    geom_point(aes(x = seq(time_range[1],(time_range[2]), by=-resolution), y = Predicted, color = "Predicted"), size = 1) +
    geom_vline(xintercept=(time_range[2] + kyrs_to_forecast), linetype="dashed", color = "red") +
    ggtitle(paste("Observed and predicted", data_to_forecast)) +
    theme(plot.title = element_text(size = 20, hjust = .5)) +
    labs(y = data_to_forecast,
         x = "age (ka BP)",
         color = NULL,
         fill = NULL) +  # Remove legend titles for color and fill
    scale_color_manual(values = c("Observed" = "red", "Predicted" = "blue", "Data" = "black"),
                       labels = c("Observed" = "KF regression", "Predicted" = "Predicted")) +
    scale_fill_manual(values = "cyan") +
    theme_light() #+
    #theme(text = element_text(family = "LM Sans 10"))
} else {
  ssvs_result <- ggplot(plot_data$predicteddata, aes(x = Time)) +
    theme(legend.position = "right") +  # Removing the plot.title element
    geom_point(data=plot_data$observeddata, aes(x = age_data_to_predict, y = data_to_predict, color = "Data")) + #, linewidth = 0.5
    #geom_line(aes(x = seq(time_range[1],(time_range[2]), by=0.1), y = Observed, color = "Observed"), linewidth = 1) +
    geom_line(aes(x = seq(time_range[1],(time_range[2]), by=resolution), y = Predicted, color = "Predicted"), linewidth = 0.75) +
    geom_ribbon(aes(x = seq(time_range[1],(time_range[2]), by=resolution), ymin = Lower, ymax = Upper, fill = "Prediction Interval"), alpha = 0.5) +
    geom_point(data = plot_data$observeddata, aes(x = age_data_to_predict, y = data_to_predict, color = "Data"), alpha = 1, size = 1) +
    geom_point(aes(x = seq(time_range[1],(time_range[2]), by=resolution), y = Predicted, color = "Predicted"), size = 1) +
    geom_vline(xintercept=(time_range[2] - kyrs_to_forecast), linetype="dashed", color = "red") +
    ggtitle(paste("Observed and predicted", data_to_forecast)) +
    theme(plot.title = element_text(size = 20, hjust = .5)) +
    labs(y = data_to_forecast,
         x = "age (ka BP)",
         color = NULL,
         fill = NULL) +  # Remove legend titles for color and fill
    scale_color_manual(values = c("Observed" = "red", "Predicted" = "blue", "Data" = "black"),
                       labels = c("Observed" = "KF regression", "Predicted" = "Predicted")) +
    scale_fill_manual(values = "cyan") +
    theme_light() #+
    #theme(text = element_text(family = "LM Sans 10"))
}



print(ssvs_result)

# Posterior predictive check

In [ ]:
# Extract Y_rep samples from JAGS output
Y_rep_samples <- as.matrix(samples[, grep("Y_rep", varnames(samples))])

# Back-transform Y_rep samples to original scale
Y_rep_original1 <- sweep(Y_rep_samples, 2, Y_sd_val, "*")
Y_rep_original <- sweep(Y_rep_original1, 2, Y_mean, "+")

# Create data frame for plotting
ppc_data <- data.frame(
  Observed = Y_original[!is.na(Y_original)],  # Remove NA values
  Replicated = apply(Y_rep_original, 2, mean)  # Take first replicate for density plot
)

# Create PPC plot
ppc_y_afterssvs <- ggplot() +
  geom_density(data = ppc_data, aes(x = Observed, fill = "Observed"),
              alpha = 0.3) +
  geom_density(data = data.frame(y = as.vector(Y_rep_original)),
               aes(x = y, fill = "Replicated"),
               alpha = 0.3) +
  scale_fill_manual(values = c("Observed" = "blue", "Replicated" = "red"),
                    name = "Data") +
  labs(x = "Values",
       y = "Density",
       title = "Posterior Predictive Check",
       subtitle = "Observed vs. Replicated Data") +
  theme_light() +
  theme(#text = element_text(family = "LM Sans 10"),
        legend.position = "bottom")

# Optional: Add numerical summary
ppc_summary <- data.frame(
  Statistic = c("Mean", "SD", "25th Percentile", "Median", "75th Percentile"),
  Observed = c(
    mean(ppc_data$Observed),
    sd(ppc_data$Observed),
    quantile(ppc_data$Observed, 0.25),
    median(ppc_data$Observed),
    quantile(ppc_data$Observed, 0.75)
  ),
  Replicated = c(
    mean(as.vector(Y_rep_original)),
    sd(as.vector(Y_rep_original)),
    quantile(as.vector(Y_rep_original), 0.25),
    median(as.vector(Y_rep_original)),
    quantile(as.vector(Y_rep_original), 0.75)
  )
)

# Print plot and summary
print(ppc_y_afterssvs)
print(ppc_summary)

# Convergence check

In [ ]:
# Trace and histogram plots
mcmc_trace_ssvs <- mcmc_trace(jags_regression_model$mcmc, pars =  c("sigma_ss", "sigma_y", "sigma_x")) + #, "sigma_x"
  ggtitle(paste("SSVS Regression", time_range[1],"-", time_range[2])) +
  theme_light() +
  #theme(text=element_text(family="LM Sans 10")) +
  theme(plot.title = element_text(size = 20, hjust = .5)) +
  theme(legend.position="none", axis.text = element_text(size = 10), axis.title = element_text(size = 11)) +
  theme(axis.title.x = element_blank(),
        axis.text.x = element_blank(), axis.ticks.x=element_blank(), axis.ticks.y=element_blank(), axis.text.y= element_blank())

print(mcmc_trace_ssvs)

In [ ]:
# Get the total number of beta parameters
n_beta <- length(covariatenames)

# Generate parameter names for all beta parameters
beta_parameters <- paste0("beta[", 1:n_beta, "]")

# Loop through all beta parameters and create density plots
mcmc_density_beta_list <- lapply(1:n_beta, function(i) {
  mcmc_dens_overlay(jags_regression_model$mcmc, pars = beta_parameters[i]) +
    ggtitle(paste("Density plot for", covariatenames[i])) +
    theme(plot.title = element_text(size = 10, hjust = .5)) +
    theme(legend.position = "none",
          axis.text = element_text(size = 8),
          axis.title = element_text(size = 9)) #+
    #theme(text = element_text(family = "LM Sans 10"))
})

# Combine all plots into a single layout using patchwork
mcmc_density_beta_combined <- wrap_plots(mcmc_density_beta_list, ncol = 4) +
  plot_annotation(title = "Density plots for Beta parameters",
                  theme = theme(plot.title = element_text(size = 15, hjust = .5))) #, family="LM Sans 10"

# Display the combined plot
print(mcmc_density_beta_combined)

# Plotting the inclusion probabilities

In [ ]:
# Remove "_mean" from the column names
names_without_mean <- gsub("_mean", "", names(X))

rename_elements <- function(name) {
  # Split the name by "[" to get the prefix and index
  parts <- unlist(strsplit(name, "\\["))
  prefix <- parts[1]
  index <- parts[2]
  index <- substr(index, 1, nchar(index) - 1)  # Remove the closing bracket

  # Create the new name based on the prefix and index
  new_name <- paste(prefix, names_without_mean[as.numeric(index)], sep = ".")
  return(new_name)
}

# Get the summary of the model_run_ssvs
model_summary <- summary(jags_regression_model)

# Get the row names of the summary object
rownames_summary <- rownames(model_summary)

# Rename each row name
new_row_names <- sapply(rownames_summary, rename_elements)


# Assign the new row names to the summary object
rownames(model_summary) <- new_row_names

# Print the modified summary
#print(model_summary)

# Extract gamma means from the model summary
gamma_means <- model_summary[grep("^gamma", rownames(model_summary)), "Mean"]
gamma_names <- names_without_mean
#

# Create a data frame for ggplot
gamma_data <- data.frame(gamma_index = gamma_names, mean_value = gamma_means)

gamma_data$gamma_label <- sub("_median_kf$", "", gamma_data$gamma_index)  # Remove "_median_kf" suffix

# Set factor levels for gamma_label to preserve the dataset order
gamma_data$gamma_label <- factor(gamma_data$gamma_label, levels = gamma_data$gamma_label)

# Plot using ggplot
incl_prob <- ggplot(gamma_data, aes(x = gamma_label, y = mean_value)) +
  geom_bar(stat = "identity") +
  labs(title = data_to_forecast,
       x = "Gamma Index",
       y = "Inclusion probability") +
  ylim(0, 1) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)#+
  #theme(text = element_text(family = "LM Sans 10"))
  ) +
    theme_light()

print(incl_prob)


# Save the plots and the result

In [ ]:

ggsave(plotraw,
       filename = paste("raw_data_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 10, width = 10, units = "in")

ggsave(mcmc_trace_covariates_kf,
       filename = paste("KF_covariate_trace_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 5, width = 8, units = "in")


ggsave(mcmc_trace_dependent_kf,
       filename = paste("KF_dependent_trace_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 3, width = 7.5, units = "in")

ggsave(plot_result_kf,
       filename = paste("KF_results_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 10, width = 10, units = "in")

ggsave(combined_trace_plots,
       filename = paste("KF_trace_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 6, width = 8, units = "in")





In [ ]:

ggsave(incl_prob,
       filename = paste("incl_prob_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 4, width = 10, units = "in")

ggsave(ssvs_result,
       filename = paste("ssvs_result_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 5, width = 7.5, units = "in")

ggsave(ssvs_trace,
       filename = paste("ssvs_trace_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 3, width = 7.5, units = "in")

ggsave(mcmc_density_beta_combined,
       filename = paste("mcmc_density_beta_combined_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 5, width = 8, units = "in")

In [ ]:

  file1 <- paste("raw_data_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")

  file2 <- paste("KF_covariate_trace_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")

  file3 <- paste("KF_dependent_trace_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")

  file4 <- paste("KF_results_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")

  file5 <- paste("KF_trace_", data_to_forecast,"_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")





  file6 <- paste("incl_prob_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")

  file7 <- paste("ssvs_result_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")

  file8 <- paste("ssvs_trace_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")

  file9 <- paste("mcmc_density_beta_combined_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")


# Correlation of the results

In [ ]:
# Read data
LR04_data_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/",
  "refs/heads/main/Data/otherdata/LR04.csv",
  sep = ""
)
chinastack_data_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/",
  "refs/heads/main/Data/otherdata/chinastack.csv",
  sep = ""
)
medstack_data_url_text <- paste(
  "https://raw.githubusercontent.com/",
  "zboraon/proxy_impuation_SSVS/",
  "refs/heads/main/Data/otherdata/medstack.csv",
  sep = ""
)

LR04_url <- getURL(LR04_data_url_text)
chinastack_url <- getURL(chinastack_data_url_text)
medstack_url <- getURL(medstack_data_url_text)

# Load data
LR04 <- read.csv(text = LR04_url)
medstack <- read.csv(text = medstack_url)
medstack_O18 <- medstack %>%
  select(time_medstack, O18_medstack)
medstack_C13 <- medstack %>%
  select(time_medstack, C13_medstack)
chinesestack <- read.csv(text = chinastack_url)

In [ ]:
x_limits <- c()
if (timescale == "forwards") {
  x_limits[1] <- time_range[2]%/%10*10
  x_limits[2] <- time_range[1]%/%10*10+10
  verticalline <- time_range[2] + kyrs_to_forecast
} else if (timescale == "backwards") {
  x_limits[1] <- time_range[1]%/%10*10
  x_limits[2] <- time_range[2]%/%10*10+10
  verticalline <- time_range[2] - kyrs_to_forecast
}




LR04_cropped <- LR04[LR04$time_lr04 >= min(time_range) & LR04$time_lr04 <= max(time_range), ]
medstack_O18_cropped <- medstack_O18[medstack_O18$time_medstack >= min(time_range) & medstack_O18$time_medstack <= max(time_range), ]
medstack_C13_cropped <- medstack_C13[medstack_C13$time_medstack >= min(time_range) & medstack_C13$time_medstack <= max(time_range), ]
chinesestack_cropped <- chinesestack[chinesestack$age_chinastack >= min(time_range) & chinesestack$age_chinastack <= max(time_range), ]


# Plot without titles, limit x-axis, and remove x-axis labels for the first two plots
p1 <- ggplot(LR04_cropped, aes(x = time_lr04, y = O18_lr04)) +
  geom_line() +
  labs(y = "O18_lr04", x = NULL) +
  # xlim(x_limits[1], x_limits[2]) +
  # scale_y_reverse() +
  #ylim(5, 3) +
  theme_minimal() +
  geom_vline(xintercept=verticalline, linetype="dashed", color = "red") +
  theme(plot.margin = margin(0, 0, 0, 0)) +
  theme(  panel.grid.major.y = element_blank(),
          panel.grid.minor.y = element_blank(),
          panel.border = element_blank()) +
  scale_x_continuous(breaks = seq(x_limits[1], x_limits[2], 10), limits = x_limits) + # Only one scale for x
  scale_y_continuous(n.breaks = 3)+
  scale_y_reverse( ) + # Combine reverse and position limits = c(2.6, -1.2),
  theme(axis.title.x = element_blank(),
        axis.text.x = element_blank(),
        axis.ticks.x = element_blank(),
        plot.margin = margin(0, 0, 0, 0)) #+
  #theme(text = element_text(family = "LM Sans 10"))

p2 <- ggplot(medstack_O18_cropped, aes(x = time_medstack, y = O18_medstack)) +
  geom_line() +
  labs(y = "O18_medstack", x = NULL) +
  # xlim(x_limits[1], x_limits[2]) +
  # scale_y_reverse() +
  #ylim(2.6, -1.2) +
  theme_minimal() +
  geom_vline(xintercept=verticalline, linetype="dashed", color = "red") +
  theme(plot.margin = margin(0, 0, 0, 0)) +
  theme(  panel.grid.major.y = element_blank(),
          panel.grid.minor.y = element_blank(),
          panel.border = element_blank()) +
  scale_x_continuous(breaks = seq(x_limits[1], x_limits[2], 10), limits = x_limits) + # Only one scale for x
  scale_y_continuous(n.breaks = 3)+
  scale_y_reverse( position = "right") + # Combine reverse and position limits = c(2.6, -1.2),
  theme(axis.title.x = element_blank(),
        axis.text.x = element_blank(),
        axis.ticks.x = element_blank(),
        axis.title.y.left = element_blank(),
        axis.text.y.right = element_text(hjust = 1),
        #axis.title.y.right = element_blank(),
        plot.margin = margin(0, 0, 0, 0)) #+
  #theme(text = element_text(family = "LM Sans 10"))

p3 <- ggplot(medstack_C13_cropped, aes(x = time_medstack, y = C13_medstack)) +
  geom_line() +
  labs(y = "C13_medstack", x = NULL) +
  # xlim(x_limits[1], x_limits[2]) +
  # scale_y_reverse() +
  #ylim(2.6, -1.2) +
  theme_minimal() +
  geom_vline(xintercept=verticalline, linetype="dashed", color = "red") +
  theme(plot.margin = margin(0, 0, 0, 0)) +
  theme(  panel.grid.major.y = element_blank(),
          panel.grid.minor.y = element_blank(),
          panel.border = element_blank()) +
  scale_x_continuous(breaks = seq(x_limits[1], x_limits[2], 10), limits = x_limits) + # Only one scale for x
  scale_y_continuous(n.breaks = 3)+
  scale_y_reverse( ) + # Combine reverse and position limits = c(2.6, -1.2),
  theme(axis.title.x = element_blank(),
        axis.text.x = element_blank(),
        axis.ticks.x = element_blank(),
        plot.margin = margin(0, 0, 0, 0)) #+
  #theme(text = element_text(family = "LM Sans 10"))

p4 <- ggplot(chinesestack_cropped, aes(x = age_chinastack, y = O18_chinastack)) +
  geom_line() +
  labs(y = "O18_chinastack", x = "Time") +
  # xlim(x_limits[1], x_limits[2]) +
  # scale_y_reverse() +
  #ylim(-5, -11.5) +
  theme_minimal() +
  geom_vline(xintercept=verticalline, linetype="dashed", color = "red") +
  theme(plot.margin = margin(0, 0, 0, 0)) +
  theme(  panel.grid.major.y = element_blank(),
          panel.grid.minor.y = element_blank(),
          panel.border = element_blank()) +
  scale_x_continuous(breaks = seq(x_limits[1], x_limits[2], 10), limits = x_limits) + # Only one scale for x
  scale_y_continuous(n.breaks = 3)+
  scale_y_reverse( position = "right") + # Combine reverse and position limits = c(2.6, -1.2),
  theme(axis.title.x = element_blank(),
        axis.text.x = element_blank(),
        axis.ticks.x = element_blank(),
        axis.title.y.left = element_blank(),
        axis.text.y.right = element_text(hjust = 1),
        #axis.title.y.right = element_blank(),
        plot.margin = margin(0, 0, 0, 0)) #+
  #theme(text = element_text(family = "LM Sans 10"))

if (timescale == "forwards") {
  p5 <- ggplot(plot_data$predicteddata, aes(x = Time)) +
    theme(#text = element_text(family = "LM Sans 10"),
          legend.position = "right") +  # Removing the plot.title element
    geom_point(data=plot_data$observeddata, aes(x = age_data_to_predict, y = data_to_predict, color = "Data")) + #, linewidth = 0.5) +
    #geom_line(aes(x = seq(time_range[1],(time_range[2]), by=0.1), y = Observed, color = "Observed"), linewidth = 1) +
    geom_line(aes(x = seq(time_range[1],(time_range[2]), by=-resolution), y = Predicted, color = "Predicted"), linewidth = 0.75) +
    geom_ribbon(aes(x = seq(time_range[1],(time_range[2]), by=-resolution), ymin = Lower, ymax = Upper, fill = "Prediction Interval"), alpha = 0.5) +
    geom_point(data = plot_data$observeddata, aes(x = age_data_to_predict, y = data_to_predict, color = "Data"), alpha = 1, size = 1) +
    geom_point(aes(x = seq(time_range[1],(time_range[2]), by=-resolution), y = Predicted, color = "Predicted"), size = 1) +
    geom_vline(xintercept=verticalline, linetype="dashed", color = "red") +
    #ggtitle("Observed and predicted O-18") +
    scale_y_reverse() +  # Remove the limits here
    coord_cartesian(ylim = c()) +  # Add this line to clip instead of remove
    scale_x_continuous(breaks = seq(x_limits[1], x_limits[2], 10), limits = x_limits) +
    #xlim(x_limits[1], x_limits[2]) +
    #ylim(6,-4) +
    theme_minimal() +

    theme(plot.title = element_text(size = 20, hjust = .5)) +
    theme(plot.margin = margin(0, 0, 0, 0)) +
    theme(  panel.grid.major.y = element_blank(),
            panel.grid.minor.y = element_blank(),
            panel.border = element_blank()) +
    labs(y = data_to_forecast,
         x = "age (ka BP)",
         color = NULL,
         fill = NULL) +  # Remove legend titles for color and fill
    scale_color_manual(values = c("Observed" = "red", "Predicted" = "blue", "Data" = "black"),
                       labels = c("Observed" = "KF regression", "Predicted" = "Predicted")) +
    scale_fill_manual(values = "cyan") +

    theme(plot.margin = margin(0, 0, 0, 0)) #+
  #theme(text = element_text(family = "LM Sans 10"))
} else {
  p5 <- ggplot(plot_data$predicteddata, aes(x = Time)) +
    theme(#text = element_text(family = "LM Sans 10"),
          legend.position = "right") +  # Removing the plot.title element
    geom_point(data=plot_data$observeddata, aes(x = age_data_to_predict, y = data_to_predict, color = "Data")) + #, linewidth = 0.5) +
    #geom_line(aes(x = seq(time_range[1],(time_range[2]), by=0.1), y = Observed, color = "Observed"), linewidth = 1) +
    geom_line(aes(x = seq(time_range[1],(time_range[2]), by=resolution), y = Predicted, color = "Predicted"), linewidth = 0.75) +
    geom_ribbon(aes(x = seq(time_range[1],(time_range[2]), by=resolution), ymin = Lower, ymax = Upper, fill = "Prediction Interval"), alpha = 0.5) +
    geom_point(data = plot_data$observeddata, aes(x = age_data_to_predict, y = data_to_predict, color = "Data"), alpha = 1, size = 1) +
    geom_point(aes(x = seq(time_range[1],(time_range[2]), by=resolution), y = Predicted, color = "Predicted"), size = 1) +
    geom_vline(xintercept=verticalline, linetype="dashed", color = "red") +
    #ggtitle("Observed and predicted O-18") +
    scale_y_reverse() +  # Remove the limits here
    coord_cartesian(ylim = c()) +  # Add this line to clip instead of remove
    scale_x_continuous(breaks = seq(x_limits[1], x_limits[2], 10), limits = x_limits) +
    #xlim(x_limits[1], x_limits[2]) +
    # ylim(5,-5) +
    theme_minimal() +

    theme(plot.title = element_text(size = 20, hjust = .5)) +
    theme(plot.margin = margin(0, 0, 0, 0)) +
    theme(  panel.grid.major.y = element_blank(),
            panel.grid.minor.y = element_blank(),
              panel.border = element_blank()) +
    labs(y = data_to_forecast,
         x = "age (ka BP)",
         color = NULL,
         fill = NULL) +  # Remove legend titles for color and fill
    scale_color_manual(values = c("Observed" = "red", "Predicted" = "blue", "Data" = "black"),
                       labels = c("Observed" = "KF regression", "Predicted" = "Predicted")) +
    scale_fill_manual(values = "cyan") +

    theme(plot.margin = margin(0, 0, 0, 0)) # +
  #theme(text = element_text(family = "LM Sans 10"))
}


# Combine plots using patchwork, remove spacing between plots, and put time axis only in the last plot
combined_plot <- (p1 / p2 / p3 / p4 /p5) +
  plot_layout(ncol = 1, heights = c(1, 1, 1, 1, 4)) &
  theme(panel.spacing = unit(0, "lines"))

# Print the combined plot
print(combined_plot)

ggsave(combined_plot,
       filename = paste("combined_plot","_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = ""),
       device = "pdf",
       height = 10, width = 10, units = "in")

file10 <- paste("combined_plot","_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".pdf", sep = "")

In [ ]:
nameworkspace <- paste("workspace", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".Rdata", sep = "")
save.image(file=paste("workspace", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".Rdata", sep = ""))


In [ ]:
if (timescale == "forwards") {
my_files <- paste("results_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".zip", sep = "")
} else {
my_files <- paste("results_", data_to_forecast,"_", mymodel, "_", time_range[1],"_", time_range[2],"_", kyrs_to_forecast, "_", resolution,".zip", sep = "")
}
zip::zip(zipfile = my_files,
        files = c(file1, file2, file3, file4, file5, file6, file7, file8, file9, file10,
        nameworkspace
        ))

